In [1]:
%autosave 120
import os.path
import numpy as np
import pandas as pd
%matplotlib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

Autosaving every 120 seconds
Using matplotlib backend: TkAgg


In [45]:
dataset_ver_walk = input('Dataset Version number (Walk): ')
walk_dataset = pd.read_csv(f'./sample_datasets/Sample_walk_dataset_{dataset_ver_walk}.csv')
#test_walk_dataset = pd.read_csv(f'./sample_datasets/Test_walk_dataset_{dataset_ver_walk}.csv')

Dataset Version number (Walk): 5


In [46]:
dataset_ver_run = input('Dataset Version number (Run): ')
run_dataset = pd.read_csv(f'./sample_datasets/Sample_run_dataset_{dataset_ver_run}.csv')
#test_run_dataset = pd.read_csv(f'./sample_datasets/Test_run_dataset_{dataset_ver_run}.csv')

Dataset Version number (Run): 5


In [47]:
dataset_ver_random = input('Dataset Version number (Random): ')
random_dataset = pd.read_csv(f'./sample_datasets/Sample_random_dataset_{dataset_ver_run}.csv')

Dataset Version number (Random): 5


In [5]:
X_indices = [1,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28]
y_indices = [2,3,4]
def count_spikes(list,TS):
    std_spike_val = 8
    diff_ts_spike = 1.0
    prev_spike_ts = 0.0
    counter = 0
    for idx,val in enumerate(list):
        if val >= std_spike_val and TS[idx] - prev_spike_ts >= diff_ts_spike:
            counter+=1
            prev_spike_ts = TS[idx]
        
    print(f'Number of spikes: {counter}')

def plot_one_plot():
    val = input('Which section? (X,Y,Z,Acc): ')
    if val == 'X':
        plt.plot(TS,X,color="red",linewidth=1.0)
    elif val == 'Y':
        plt.plot(TS,Y,color="red",linewidth=1.0)
    elif val == 'Z':
        plt.plot(TS,Z,color="red",linewidth=1.0)
    else:
        plt.axhline(y=12.5)
        plt.axhline(y=10)
        plt.scatter(TS,Acc,s=10,color="blue")
        plt.plot(TS,Acc,color="red",linewidth=1.0)
    plt.xlabel('Time Stamp')
    plt.ylabel('acceleration in m/s (Gravity Incl)')


def plot_sub_plots():
    figure, axis = plt.subplots(2, 2)

    axis[0,0].plot(TS,X,color="red",linewidth=1.0)
    axis[0,0].set_title("WRT X Values")

    axis[0,1].plot(TS,Y,color="blue",linewidth=1.0)
    axis[0,1].set_title("WRT Y Values")

    axis[1,0].plot(TS,Z,color="yellow",linewidth=1.0)
    axis[1,0].set_title("WRT Z Values")

    axis[1,1].plot(TS,X,color="red",linewidth=1.0)
    axis[1,1].plot(TS,Y,color="blue",linewidth=1.0)
    axis[1,1].plot(TS,Z,color="yellow",linewidth=1.0)
    axis[1,1].plot(TS,Acc,color="grey",linewidth=1.0)
    axis[1,1].set_title("WRT X,Y,Z,ACC Values")
#    [1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26] 
def preprocessForXandY(dataset_walk, dataset_run,dataset_rand):
    X_walk = dataset_walk.iloc[:,X_indices]
    y_walk = dataset_walk.iloc[:,y_indices]
    X_run = dataset_run.iloc[:,X_indices]
    y_run = dataset_run.iloc[:,y_indices]
    X_rand = dataset_rand.iloc[:,X_indices]
    y_rand = dataset_rand.iloc[:,y_indices]
    
    X_frames = [X_walk,X_run,X_rand]
    X = pd.concat(X_frames)
    Y_frames = [y_walk,y_run,y_rand]
    y = pd.concat(Y_frames)
    return X,y

def preprocessAndSplit(dataset_walk, dataset_run, dataset_rand):
    W_train, W_test, Wy_train, Wy_test = train_test_split(
        dataset_walk.iloc[:,X_indices], dataset_walk.iloc[:,y_indices], test_size=0.25, shuffle=False, stratify=None)
    R_train, R_test, Ry_train, Ry_test = train_test_split(
        dataset_run.iloc[:, X_indices], dataset_run.iloc[:,y_indices], test_size=0.25, shuffle=False, stratify=None)
    Rd_train, Rd_test, Rdy_train, Rdy_test = train_test_split(
        dataset_rand.iloc[:,X_indices], dataset_rand.iloc[:,y_indices], test_size=0.25, shuffle=False, stratify=None)
    
    X_train = pd.concat([W_train,R_train,Rd_train])
    y_train = pd.concat([Wy_train,Ry_train,Rdy_train])
    X_test = pd.concat([W_test,R_test,Rd_test])
    y_test = pd.concat([Wy_test,Ry_test,Rdy_test])
    return X_train,y_train,X_test,y_test

def makeOneObservation(input_dataset):
    row,col = total_observations = input_dataset.shape
    print(row,col)

    new_observation = []
    ts_i = 0
    for ts_i in range(0,row-3,4):
        new_observation.append([
            input_dataset.iloc[ts_i,0],
                       input_dataset.iloc[ts_i,1],
                       input_dataset.iloc[ts_i,2],input_dataset.iloc[ts_i,3],input_dataset.iloc[ts_i,4],
                       input_dataset.iloc[ts_i,5], input_dataset.iloc[ts_i,6],input_dataset.iloc[ts_i,7],
                       input_dataset.iloc[ts_i,8], input_dataset.iloc[ts_i,9],input_dataset.iloc[ts_i,10],
                       
                       input_dataset.iloc[ts_i+1,5], input_dataset.iloc[ts_i+1,6],input_dataset.iloc[ts_i+1,7],
                       input_dataset.iloc[ts_i+1,8], input_dataset.iloc[ts_i+1,9],input_dataset.iloc[ts_i+1,10],
                    
                       input_dataset.iloc[ts_i+2,5], input_dataset.iloc[ts_i+2,6],input_dataset.iloc[ts_i+2,7],
                       input_dataset.iloc[ts_i+2,8], input_dataset.iloc[ts_i+2,9],input_dataset.iloc[ts_i+2,10],
                       
                       input_dataset.iloc[ts_i+3,5], input_dataset.iloc[ts_i+3,6],input_dataset.iloc[ts_i+3,7],
                       input_dataset.iloc[ts_i+3,8], input_dataset.iloc[ts_i+3,9],input_dataset.iloc[ts_i+3,10],
                       ])
        
    new_obs = pd.DataFrame(new_observation,columns=['TS','Age','Random','Walk','Run',
                                'Xa_1','Ya_1','Za_1','Xg_1','Yg_1','Zg_1',
                                'Xa_2','Ya_2','Za_2','Xg_2','Yg_2','Zg_2',
                                'Xa_3','Ya_3','Za_3','Xg_3','Yg_3','Zg_3',
                                'Xa_4','Ya_4','Za_4','Xg_4','Yg_4','Zg_4'
                                ])
    return new_obs

def accuracyNums(y_exp, y_pred):
    model_accuracy = accuracy_score(y_exp, y_pred)
    return (model_accuracy,confusion_matrix(y_exp,y_pred))

def saveMyModel(model,version_num):
    file_name = f'trained_models/run_walk_random_model_v{version_num}'
    if os.path.isfile(f'{file_name}.h5') is False:
        model.save(f'{file_name}.h5')
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()
    with open(f'{file_name}.tflite', 'wb') as f:
      f.write(tflite_model)
    interpreter = tf.lite.Interpreter(model_path=f'{file_name}.tflite')
    interpreter.allocate_tensors()

    print(f'Model saved as: {file_name} in h5 and tflite')
    # Print input shape and type
    inputs = interpreter.get_input_details()
    print('{} input(s):'.format(len(inputs)))
    for i in range(0, len(inputs)):
        print('{} {}'.format(inputs[i]['shape'], inputs[i]['dtype']))

    # Print output shape and type
    outputs = interpreter.get_output_details()
    print('\n{} output(s):'.format(len(outputs)))
    for i in range(0, len(outputs)):
        print('{} {}'.format(outputs[i]['shape'], outputs[i]['dtype']))

In [ ]:
choice = input('run or walk? ')
if choice == 'run':
    matrix = run_dataset
else:
    matrix = walk_dataset
plot_type = input('What type of plot? Single (S) or Multiple (M) or count spikes (CS): ')
#LR = matrix.iloc[:,[0]].values
RW = matrix.iloc[:,[2]].values
X = matrix.iloc[:,[3]].values
Y = matrix.iloc[:,[4]].values
Z = matrix.iloc[:,[5]].values
Acc = matrix.iloc[:,[9]].values
#TS_i = matrix.iloc[:,[4]].values[0]
#TS = (matrix.iloc[:,[4]].values - TS_i)/100000000
TS = matrix.iloc[:,[0]].values
if TS[0] > 100000000:
    TS = convert_ts(TS)
if plot_type == 'S':
    plot_one_plot()
elif plot_type == 'M': 
    plot_sub_plots()
else:
    count_spikes(Acc,TS=TS)
plt.show()

In [ ]:
# new_data = makeOneObservation(run_dataset)
# new_data

In [48]:
walk_dataset = makeOneObservation(walk_dataset)
run_dataset = makeOneObservation(run_dataset)
random_dataset = makeOneObservation(random_dataset)
X,y = preprocessForXandY(walk_dataset,run_dataset,random_dataset)
X_train,y_train,X_test,y_test = preprocessAndSplit(walk_dataset,run_dataset,random_dataset)

print(f'{X} \n {y} \n {X_train} \n {y_train} \n {X_test} \n {y_test}')

26 12
33 12
23 12
   Age  Xa_1  Ya_1  Za_1  Xg_1  Yg_1  Zg_1  Xa_2  Ya_2  Za_2  ...  Za_3  Xg_3  \
0   21   0.1   0.0   0.1   0.0   0.0   0.0  -0.6   0.7   0.4  ...  -0.2  -0.7   
1   21  -0.5   0.0   3.0  -0.8   0.0  -0.2  -0.2   4.0   5.8  ...  -0.6   0.3   
2   21  -1.7  -0.8  -0.8   1.0   0.4   0.1   1.3  -1.5  -3.7  ...   0.2   1.1   
3   21  -2.9   1.7   0.3   0.8  -0.1  -0.1   0.4   0.0   0.4  ...  -0.5   0.0   
4   21   1.5  -1.9   0.5  -0.9  -0.3  -0.3   2.3   0.5  -1.1  ...  -2.8  -1.1   
5   21   1.1  -1.1  -0.3  -0.5   0.1  -0.1   0.2   0.1   2.0  ...  -3.5   0.5   
0   21   0.7   2.1  -0.4  -1.8  -0.8   0.0  -2.8   2.0   4.9  ...  -5.9   2.2   
1   21  -1.0   4.3   1.6  -4.5  -0.2  -0.3  -4.9   4.3   3.3  ...  -3.3   2.2   
2   21   4.4  -3.7  -0.5  -2.9  -0.1  -0.2   6.8 -10.4  -3.0  ...   4.9   1.5   
3   21   0.5  -1.5  12.3  -0.1  -0.3  -0.3  -1.8   1.7  -6.0  ...   4.9  -0.1   
4   21  -5.7   5.3   3.8   2.8   0.3  -0.6   4.4   2.6  -6.6  ...   3.2  -2.4   
5   21   3

In [26]:
X.columns

Index(['Age', 'Xa_1', 'Ya_1', 'Za_1', 'Xg_1', 'Yg_1', 'Zg_1', 'Xa_2', 'Ya_2',
       'Za_2', 'Xg_2', 'Yg_2', 'Zg_2', 'Xa_3', 'Ya_3', 'Za_3', 'Xg_3', 'Yg_3',
       'Zg_3', 'Xa_4', 'Ya_4', 'Za_4', 'Xg_4', 'Yg_4', 'Zg_4'],
      dtype='object')

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

In [49]:
model_v = input('Version: ')
file_name = f'trained_models/run_walk_random_model_v{model_v}.h5'
model = keras.models.load_model(file_name)

Version: 1.1


In [8]:
model = Sequential([
    Dense(units=32, input_shape=(25,), activation='relu'),
    Dense(units=64, activation='relu'),
    Dense(units=3, activation='softmax')
])

In [9]:
model.summary()
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                832       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 195       
Total params: 3,139
Trainable params: 3,139
Non-trainable params: 0
_________________________________________________________________


In [50]:
model.fit(x=X_train,y=y_train, batch_size=32,epochs=50, shuffle=False,verbose=2)

Epoch 1/50
1/1 - 0s - loss: 0.5231 - accuracy: 0.7692
Epoch 2/50
1/1 - 0s - loss: 0.4553 - accuracy: 0.7692
Epoch 3/50
1/1 - 0s - loss: 0.3617 - accuracy: 0.7692
Epoch 4/50
1/1 - 0s - loss: 0.2754 - accuracy: 0.9231
Epoch 5/50
1/1 - 0s - loss: 0.2214 - accuracy: 0.9231
Epoch 6/50
1/1 - 0s - loss: 0.1949 - accuracy: 1.0000
Epoch 7/50
1/1 - 0s - loss: 0.1822 - accuracy: 1.0000
Epoch 8/50
1/1 - 0s - loss: 0.1760 - accuracy: 1.0000
Epoch 9/50
1/1 - 0s - loss: 0.1727 - accuracy: 1.0000
Epoch 10/50
1/1 - 0s - loss: 0.1681 - accuracy: 1.0000
Epoch 11/50
1/1 - 0s - loss: 0.1629 - accuracy: 1.0000
Epoch 12/50
1/1 - 0s - loss: 0.1548 - accuracy: 1.0000
Epoch 13/50
1/1 - 0s - loss: 0.1448 - accuracy: 1.0000
Epoch 14/50
1/1 - 0s - loss: 0.1344 - accuracy: 1.0000
Epoch 15/50
1/1 - 0s - loss: 0.1248 - accuracy: 1.0000
Epoch 16/50
1/1 - 0s - loss: 0.1158 - accuracy: 1.0000
Epoch 17/50
1/1 - 0s - loss: 0.1072 - accuracy: 1.0000
Epoch 18/50
1/1 - 0s - loss: 0.0991 - accuracy: 1.0000
Epoch 19/50
1/1 - 0

In [38]:
# new_walk_dataset = makeOneObservation(pd.read_csv(f'./sample_datasets/Sample_walk_dataset_main_1.csv'))
# new_run_dataset = makeOneObservation(pd.read_csv(f'./sample_datasets/Sample_run_dataset_main_2.csv'))
# test_X,test_y = preprocessForXandY(new_walk_dataset,new_run_dataset)
# X_walk = walk_dataset.iloc[:,[1,3,4,5,6,7,8,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26]]
# y_walk = walk_dataset.iloc[:,[2]]
# X_test y_test
model_predictions = model.predict(x=X_test, batch_size=10, verbose=0)
round_pred = np.argmax(model_predictions, axis=-1)

#print(accuracyNums(y_test,round_pred))
model_predictions

array([[4.0762302e-01, 5.7514119e-01, 1.7235762e-02],
       [1.8840061e-01, 7.9322731e-01, 1.8372096e-02],
       [9.9985776e-03, 5.3163284e-01, 4.5836857e-01],
       [1.7453098e-06, 9.9881315e-01, 1.1850977e-03],
       [5.0050175e-01, 4.9375600e-01, 5.7422956e-03],
       [6.4066428e-01, 3.5067278e-01, 8.6630378e-03]], dtype=float32)

In [56]:
# new_walk_dataset = pd.read_csv(f'./Sample Datasets/Sample_walk_dataset_3.csv')
# new_run_dataset = pd.read_csv(f'./Sample Datasets/Sample_run_dataset_3.csv')
# X,y = preprocessForXandY(new_walk_dataset,new_run_dataset)
# preds = model.predict(X)
# preds
print(model.predict([[21,
                      1.9,-1.5,0.9,-0.9,-0.9,-0.6
,4.4,-6.3,-0.1,0.9,0.3,0.3
,-1.7,-1.7,2.2,-0.7,-1,-0.4
,-5.7,1.3,1.5,0.7,0.5,6]]))



# print(model.predict([[0.4,1.4,-6.9],
# [-0.9,1.2,-2.2],
# [-2.1,1.8,1.9],
# [-1.8,3,2.2],
# [1.5,3.7,2.2]]))

[[0.21486369 0.7602721  0.0248642 ]]


In [51]:
saveMyModel(model,input('Model Version No. '))

Model Version No. 1.2
INFO:tensorflow:Assets written to: C:\Users\Mihir\AppData\Local\Temp\tmpxtzx9kl1\assets


INFO:tensorflow:Assets written to: C:\Users\Mihir\AppData\Local\Temp\tmpxtzx9kl1\assets


Model saved as: trained_models/run_walk_random_model_v1.2 in h5 and tflite
1 input(s):
[ 1 25] <class 'numpy.float32'>

1 output(s):
[1 3] <class 'numpy.float32'>


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier()

In [ ]:
KNN.fit(X, y)

In [ ]:
new_walk_dataset = pd.read_csv(f'./sample_datasets/Sample_walk_dataset_3.csv')
new_run_dataset = pd.read_csv(f'./sample_datasets/Sample_run_dataset_1.csv')
test_X,test_y = preprocessForXandY(new_walk_dataset,new_run_dataset)
prediction_KNN = KNN.predict(test_X)
accuracy_knn = accuracy_score(test_y, prediction_KNN)
print (accuracy_knn,confusion_matrix(test_y,prediction_KNN))

In [ ]:
saveMyModel(KNN,input('Model Version No. '))

In [ ]:
prediction_KNN = KNN.predict([[0.4,1.4,-6.9],
[-0.9,1.2,-2.2],
[-2.1,1.8,1.9],
[-1.8,3,2.2],
[1.5,3.7,2.2]])
prediction_KNN